# Training and Testing the Model
Lots of credit to the homework 1 code

In [86]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
import sys
import random

DATA_PATH = 'simple_images'

### Import the Dataset

In [129]:
norm_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.0, 1.0)
])

class ParksDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, path, train, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.path = path
        self.transform = transform
        self.train = train
                
        # import the data now
        self.data = []
        dirs = os.listdir(path)
        dirs.sort()
        for item in dirs:
            if not item.startswith("."):
                images = os.listdir(path + "/" + item)
                count = int(len(images) * 0.8)
                if train:
                    images = images[:count]
                else:
                    images = images[count:]
                    
                for image in images:
                    if not image.startswith("."):
                        label = item.replace("+", " ")[:-len(" Landscape")]
                        self.data.append((path + "/" + item + "/" + image, label))  
        random.shuffle(self.data)
        
        i = 0
        self.label_to_idx = {}
        for item in dirs:
            if not item.startswith("."):
                label = item.replace("+", " ")[:-len(" Landscape")]
                self.label_to_idx[label] = i
                i += 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = norm_transform(io.imread(self.data[idx][0]))
        label = self.label_to_idx[self.data[idx][1]]
        return image, label
    
train_dataset = ParksDataset(csv_file="national-parks.csv", path="simple_images", train=True)
test_dataset = ParksDataset(csv_file="national-parks.csv", path="simple_images", train=False)

### Define the Model

In [130]:
class NatParkNet(nn.Module):
    def __init__(self):
        super(NatParkNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5, stride=2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 128, 3)
        self.conv5 = nn.Conv2d(128, 256, 2)

        self.norm1 = nn.BatchNorm2d(16)
        self.norm2 = nn.BatchNorm2d(32)
        self.norm3 = nn.BatchNorm2d(64)
        self.norm4 = nn.BatchNorm2d(128)
        self.norm5 = nn.BatchNorm2d(256)

        self.maxpool = nn.MaxPool2d(2)
        self.avgpool = nn.AvgPool2d(2)

        self.fc1 = nn.Linear(1024, len(train_dataset.label_to_idx))
        self.accuracy = None

    def forward(self, x):
        x = F.relu(self.norm1(self.maxpool(self.conv1(x))))
        x = F.relu(self.norm2(self.maxpool(self.conv2(x))))
        x = F.relu(self.norm3(self.maxpool(self.conv3(x))))
        x = F.relu(self.norm4(self.maxpool(self.conv4(x))))
        x = F.relu(self.norm5(self.avgpool(self.conv5(x))))
        return self.fc1(torch.flatten(x, 1))

    def loss(self, prediction, label, reduction='mean'):
        loss_val = F.cross_entropy(prediction, label.squeeze(), reduction=reduction)
        return loss_val

### Training

In [131]:
import time
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    losses = []
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = model.loss(output, label)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('{} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                time.ctime(time.time()),
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(losses)

def test(model, device, test_loader, log_interval=None):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss_on = model.loss(output, label, reduction='sum').item()
            test_loss += test_loss_on
            pred = output.max(1)[1]
            correct_mask = pred.eq(label.view_as(pred))
            num_correct = correct_mask.sum().item()
            correct += num_correct
            if log_interval is not None and batch_idx % log_interval == 0:
                print('{} Test: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    time.ctime(time.time()),
                    batch_idx * len(data), len(test_loader.dataset),
                    100. * batch_idx / len(test_loader), test_loss_on))

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
    return test_loss, test_accuracy

In [133]:
BATCH_SIZE = 256
TEST_BATCH_SIZE = 10
EPOCHS = 20
LEARNING_RATE = 0.01
MOMENTUM = 0.9
USE_CUDA = True
SEED = 0
PRINT_INTERVAL = 100
WEIGHT_DECAY = 0.0005

DATA_PATH = os.getcwd()

EXPERIMENT_VERSION = "0.1"
LOG_PATH = DATA_PATH + 'logs/' + EXPERIMENT_VERSION + '/'

# Now the actual training code
use_cuda = USE_CUDA and torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)
import multiprocessing
print('num cpus:', multiprocessing.cpu_count())

kwargs = {'num_workers': multiprocessing.cpu_count(),
          'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE,
                                          shuffle=False, **kwargs)

model = NatParkNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
start_epoch = 0

train_losses = []
test_losses = []
test_accuracies = []
test_loss, test_accuracy = test(model, device, test_loader)

test_losses.append((start_epoch, test_loss))
test_accuracies.append((start_epoch, test_accuracy))

for epoch in range(start_epoch, EPOCHS + 1):
    train_loss = train(model, device, train_loader, optimizer, epoch, PRINT_INTERVAL)
    test_loss, test_accuracy = test(model, device, test_loader)
    train_losses.append((epoch, train_loss))
    test_losses.append((epoch, test_loss))
    test_accuracies.append((epoch, test_accuracy))

Using device cpu
num cpus: 16

Test set: Average loss: 4.1108, Accuracy: 73/3840 (2%)

Mon Dec 12 21:32:33 2022 Train Epoch: 0 [0/15253 (0%)]	Loss: 4.184405

Test set: Average loss: 3.2262, Accuracy: 734/3840 (19%)

Mon Dec 12 21:36:23 2022 Train Epoch: 1 [0/15253 (0%)]	Loss: 2.792121

Test set: Average loss: 2.5733, Accuracy: 1291/3840 (34%)

Mon Dec 12 21:40:16 2022 Train Epoch: 2 [0/15253 (0%)]	Loss: 1.981238

Test set: Average loss: 1.8119, Accuracy: 2251/3840 (59%)

Mon Dec 12 21:44:05 2022 Train Epoch: 3 [0/15253 (0%)]	Loss: 1.419944

Test set: Average loss: 1.6644, Accuracy: 2078/3840 (54%)

Mon Dec 12 21:47:51 2022 Train Epoch: 4 [0/15253 (0%)]	Loss: 0.833485

Test set: Average loss: 0.9260, Accuracy: 3110/3840 (81%)

Mon Dec 12 21:51:42 2022 Train Epoch: 5 [0/15253 (0%)]	Loss: 0.408472

Test set: Average loss: 0.4315, Accuracy: 3621/3840 (94%)

Mon Dec 12 21:55:32 2022 Train Epoch: 6 [0/15253 (0%)]	Loss: 0.177558

Test set: Average loss: 0.2435, Accuracy: 3746/3840 (98%)

Mon 

In [134]:
# Save out the model
torch.save(model.state_dict(), 'my_model')

In [ ]:
import matplotlib.pyplot as plt

# Show some results
plt.plot(np.array(test_accuracies)[:,1])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Test Accuracy")
plt.savefig("test_acc.png", dpi=300, format="png")

In [151]:
from sklearn.metrics import confusion_matrix

# Some code to show a confusion matrix, turns out not very interesting with a good model
true = torch.zeros(0, dtype=torch.long)
pred = torch.zeros(0, dtype=torch.long)

with torch.no_grad():
    for i, (inputs, classes) in enumerate(train_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)

        true = torch.cat([true, classes.view(-1)])
        pred = torch.cat([pred, predictions.view(-1)])


# Confusion matrix and accuracy
confusion = confusion_matrix(true.numpy(), pred.numpy())
print(confusion)
accuracy = 100 * confusion.diagonal() / confusion.sum(1)
print(accuracy)

[[294   0   0 ...   0   0   0]
 [  0 272   0 ...   0   0   0]
 [  0   0 264 ...   0   0   0]
 ...
 [  0   0   0 ... 280   0   0]
 [  0   0   0 ...   0 286   0]
 [  0   0   0 ...   0   0 284]]
[100.         100.         100.         100.         100.
 100.         100.         100.         100.         100.
 100.         100.          98.94366197 100.         100.
 100.         100.         100.         100.         100.
  98.33887043 100.         100.         100.         100.
 100.         100.         100.         100.          98.12030075
  97.6744186  100.          96.77419355 100.         100.
 100.         100.         100.         100.         100.
 100.          99.64788732 100.         100.         100.
  89.12280702 100.         100.         100.         100.
 100.         100.          96.55172414 100.         100.        ]
